In [ ]:
import pandas as pd
import mailbox
from email import message_from_string
from tqdm import tqdm

# Load mbox file
mbox_file_path = '/content/drive/MyDrive/sample.mbox'
mbox = mailbox.mbox(mbox_file_path)

# Extract messages and their IDs
messages = {}
select_date = '2023-06-06'  # Replace with your desired date
select_date = pd.to_datetime(select_date).tz_localize('UTC')  # Convert select_date to UTC

for i, message in tqdm(enumerate(mbox), desc='Extracting messages', total=len(mbox)):
    message_id = message.get('Message-ID')
    message_date = pd.to_datetime(message['Date'])
    if message_date.tzinfo is not None:
        message_date = message_date.tz_convert('UTC')
    else:
        message_date = message_date.tz_localize('UTC')

    if message_date > select_date:
        print("yes")
        messages[message_id] = {
            'Subject': message['Subject'],
            'From': message['From'],
            'To': message['To'],
            'Date': message['Date'],
            'Body': message.get_payload(),
            'References': message.get('References', '').split(),
            'In-Reply-To': message.get('In-Reply-To', '')
        }


# Reconstruct conversation threads
threads = {}
for message_id, message_data in tqdm(messages.items(), desc='Reconstructing threads'):
    thread_id = message_id
    while message_data['In-Reply-To'] in messages:
        thread_id = message_data['In-Reply-To']
        message_data = messages[thread_id]

    if thread_id not in threads:
        threads[thread_id] = {
            'ThreadID': thread_id,
            'Messages': []
        }

    threads[thread_id]['Messages'].append(message_id)

# Extract email bodies and concatenate them within threads
for thread_id, thread_data in tqdm(threads.items(), desc='Extracting email bodies'):
    email_bodies = []
    for msg_id in thread_data['Messages']:
        message = messages[msg_id]
        body = ''
        if isinstance(message['Body'], str):
            body = message['Body']
        elif isinstance(message['Body'], list):
            for part in message['Body']:
                if part.get_content_type() == 'text/plain':
                    body += part.get_payload()
        email_bodies.append(body)

    thread_data['email_body'] = '\n'.join(email_bodies)

# Convert threads to DataFrame
thread_list = [thread for thread_id, thread in threads.items()]
df = pd.DataFrame(thread_list)

# Display the DataFrame
print(df.head())

In [ ]:
def process_mbox(file_path, select_date):
    mbox = mailbox.mbox(file_path)

    # Extract messages and their IDs
    messages = {}
    select_date = pd.to_datetime(select_date).tz_localize('UTC')  # Convert select_date to UTC

    for i, message in tqdm(enumerate(mbox), desc='Extracting messages', total=len(mbox)):
        message_id = message.get('Message-ID')
        message_date = pd.to_datetime(message['Date'])
        if message_date.tzinfo is not None:
            message_date = message_date.tz_convert('UTC')
        else:
            message_date = message_date.tz_localize('UTC')

        if message_date > select_date:
            messages[message_id] = {
                'Subject': message['Subject'],
                'From': message['From'],
                'To': message['To'],
                'Date': message['Date'],
                'Body': message.get_payload(),
                'References': message.get('References', '').split(),
                'In-Reply-To': message.get('In-Reply-To', '')
            }

    # Reconstruct conversation threads
    threads = {}
    for message_id, message_data in tqdm(messages.items(), desc='Reconstructing threads'):
        thread_id = message_id
        while message_data['In-Reply-To'] in messages:
            thread_id = message_data['In-Reply-To']
            message_data = messages[thread_id]

        if thread_id not in threads:
            threads[thread_id] = {
                'ThreadID': thread_id,
                'Messages': []
            }

        threads[thread_id]['Messages'].append(message_id)

    # Extract email bodies and concatenate them within threads
    for thread_id, thread_data in tqdm(threads.items(), desc='Extracting email bodies'):
        email_bodies = []
        for msg_id in thread_data['Messages']:
            message = messages[msg_id]
            body = ''
            if isinstance(message['Body'], str):
                body = message['Body']
            elif isinstance(message['Body'], list):
                for part in message['Body']:
                    if part.get_content_type() == 'text/plain':
                        body += part.get_payload()
            email_bodies.append(body)

        thread_data['email_body'] = '\n'.join(email_bodies)

    # Convert threads to DataFrame
    thread_list = [thread for thread_id, thread in threads.items()]
    df = pd.DataFrame(thread_list)

    return df



In [ ]:
# Example usage:
mbox_file_path = '/content/drive/MyDrive/sample.mbox' #your mbox file path
result_df = process_mbox(mbox_file_path, date_to_select)
result_df
